PyBaMM now supports loading third-party models and parameter sets via a new public API:

- pybamm.Model("model_name")
- pybamm.ParameterValues("parameter_set_name")

These allow users to access models **distributed as Python packages** using Python's entry_points mechanism.



In this notebook, we’ll demonstrate how to use this API with an external package: [pybamm-reservoir-example](https://github.com/pybamm-team/pybamm-reservoir-example), which provides a simple reservoir model and parameter set.

If you want to create your own external model package, we recommend starting with pybamm-cookie, a cookiecutter template that helps you set up your model as a Python package with the correct structure and metadata.


To make your models and parameter sets discoverable through PyBaMM’s public API, you need to register them under the appropriate entry point groups in your pyproject.toml. Here’s how to do that:

```
[project.entry-points."pybamm_parameter_sets"]
ReservoirParams = "pybamm_reservoir_example.parameters.input.ReservoirParameters:get_parameter_values" 
```
```

[project.entry-points."pybamm_models"] 
Reservoir = "pybamm_reservoir_example.models.input.Reservoir:Reservoir" 

```

You can install the package using `pip install .` ,if it’s available locally, or install it directly from GitHub using:

In [1]:
!pip install git+https://github.com/pybamm-team/pybamm-reservoir-example.git@main

!pip install "pybamm>=25.8"


  Cloning https://github.com/pybamm-team/pybamm-reservoir-example.git (to revision main) to /tmp/pip-req-build-wwgghxl6
  Running command git clone --filter=blob:none --quiet https://github.com/pybamm-team/pybamm-reservoir-example.git /tmp/pip-req-build-wwgghxl6
  Resolved https://github.com/pybamm-team/pybamm-reservoir-example.git to commit 679602a0294cd29ae57f890a4e1bcbfeaeccb2ce
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pybamm-reservoir-example: filename=pybamm_reservoir_example-0.1.dev8+g679602a02-py3-none-any.whl size=18707 sha256=8cdc32ebd1c7683751c901b05191f6de5c59558efc6202fcefe8ebc62c2a6225
  Stored in directory: /tmp/pip-ephem-wheel-cache-nwsotoc6/wheels/65/7f/a0/91815ca1d7c5048260515e22d2f6e925734bdf5c25110fa560
Successfully built pybamm-reservoir-example
  Attempting uninstall: pybamm-reservoir-example
    Found existing installation: pybamm-reservoir-example 0.1

Once the package is installed, you can load the model and parameter set directly using the `pybamm.Model()` and `pybamm.ParameterValues()` APIs — no manual imports from the package are required.

In [2]:
import pybamm

# Load model and parameters from entry points
model = pybamm.Model("Reservoir")
params = pybamm.ParameterValues("ReservoirParams")


In [3]:

# Create simulation and solve
sim = pybamm.Simulation(model, parameter_values=params)
sim.solve([0, 3600])
sim.plot(['Negative electrode stoichiometry', 'Positive electrode stoichiometry'])

/home/medha/anaconda3/envs/test1/lib/python3.12/site-packages/pybamm/simulation.py:122: UserWarning: The default solver changed to IDAKLUSolver after the v25.4.0. release. You can swap back to the previous default by using `pybamm.CasadiSolver()` instead.
  self._solver = solver or self._model.default_solver


interactive(children=(FloatSlider(value=0.0, description='t', max=0.895106160462667, step=0.00895106160462667)…

### You can also view the available models and parameter sets registered with PyBaMM.

In [4]:
print("Available models:")
print(list(pybamm.dispatch.models))

print("\nAvailable parameter sets:")
print(list(pybamm.parameter_sets))


Available models:
['BasicReservoir', 'Reservoir', 'SPM', 'DFN', 'MPM', 'MSMR', 'NewmanTobias', 'SPMe', 'Yang2017']

Available parameter sets:
['Chen2020', 'ReservoirParams', 'Ai2020', 'Chayambuka2022', 'Chen2020_composite', 'ECM_Example', 'Ecker2015', 'Ecker2015_graphite_halfcell', 'MSMR_Example', 'Marquis2019', 'Mohtat2020', 'NCA_Kim2011', 'OKane2022', 'OKane2022_graphite_SiOx_halfcell', 'ORegan2022', 'Prada2013', 'Ramadass2004', 'Sulzer2019', 'Xu2019']


### To understand what a model or parameter set does you can get the docstring of the model and parameter set:

In [5]:
# View docstring for a model
print(pybamm.dispatch.models.get_docstring("Reservoir"))


Simple stoichiometric reservoir model for a lithium-ion battery.

This model captures the evolution of stoichiometries in the negative and
positive electrodes due to applied current, using a lumped approach with
simplified voltage dynamics.

Parameters
----------
name : str, optional
    The name of the model.



In [6]:
# View docstring for a parameter set
print(pybamm.parameter_sets.get_docstring("ReservoirParams"))


Parameter values for a simple reservoir model using LG M50 cell data.

The OCV curves are taken from the paper :footcite:t:`Chen2020`, while
other parameter values are illustrative. These include a sinusoidal
current function, initial stoichiometries, and electrode properties.

.. note::
    This parameter set is intended for use with the `Reservoir` model,
    and is not meant to be representative of detailed cell-level physics.

Returns
-------
dict
    Dictionary of parameter values compatible with `pybamm.ParameterValues`.

